In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# Change working directory to /src


import os
from v4vapp_backend_v2.database.db import MongoDBClient
from v4vapp_backend_v2.config.setup import InternalConfig
import os
import pandas as pd
import matplotlib.pyplot as plt
from v4vapp_backend_v2.database.db import get_mongodb_client_defaults
from v4vapp_backend_v2.actions.tracked_models import TrackedBaseModel

import os

target_dir = "/Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2/"
os.chdir(target_dir)
print("Current working directory:", os.getcwd())


Current working directory: /Users/bol/Documents/dev/v4vapp/v4vapp-backend-v2


In [3]:
from v4vapp_backend_v2.accounting.balance_sheet import (
    get_account_balance_printout,
    get_ledger_dataframe,
    list_all_accounts,
)
from v4vapp_backend_v2.accounting.account_type import Account


CONFIG = InternalConfig(config_filename="devhive.config.yaml").config

db_client = get_mongodb_client_defaults()

TrackedBaseModel.db_client = db_client
ledger_df = await get_ledger_dataframe()


all_accounts = await list_all_accounts()
print("All accounts:", all_accounts)
balances = {}
for account in all_accounts:
    print(f"Account: {account}")
    balances[str(account)] = await get_account_balance_printout(
        account=account, full_history=True, df=ledger_df
    )


2025-06-09T12:06:04+0300.444 INFO     db                        161 : 📁 backend Initializing MongoDBClient local_connection


Config file found: config/devhive.config.yaml


2025-06-09T12:06:04+0300.465 INFO     db                        557 : 📁 backend Attempting to connect to MongoDB for the first time or after failure.


All accounts: [Customer Deposits Hive (Asset) - Sub: devser.v4vapp, Escrow Hive (Asset) - Sub: devser.v4vapp, Treasury Hive (Asset) - Sub: devtre.v4vapp, Treasury Lightning (Asset) - Sub: umbrel, Customer Liability Hive (Liability) - Sub: v4vapp-test, Owner Loan Payable (funding) (Liability) - Sub: v4vapp.dhf, Fee Income Lightning (Revenue) - Sub: umbrel]
Account: Customer Deposits Hive (Asset) - Sub: devser.v4vapp
Account: Escrow Hive (Asset) - Sub: devser.v4vapp
Account: Treasury Hive (Asset) - Sub: devtre.v4vapp
Account: Treasury Lightning (Asset) - Sub: umbrel
Account: Customer Liability Hive (Liability) - Sub: v4vapp-test
Account: Owner Loan Payable (funding) (Liability) - Sub: v4vapp.dhf
Account: Fee Income Lightning (Revenue) - Sub: umbrel


In [4]:
from v4vapp_backend_v2.accounting.balance_sheet import get_ledger_entries


ledger_entries = await get_ledger_entries()
for ledger_entry in ledger_entries:
    print(ledger_entry)


J/E NUMBER: 95793083_8fc6b78d32c8c8050aae9117915b3c17757f65b6_1_real 95793083_8fc6b
DATE
May 11, 2025 09:58  

ACCOUNT                                                                 DEBIT          CREDIT
----------------------------------------------------------------------------------------------------
Treasury Hive (Asset)                           devtre.v4vapp     100.00 hive                
    Owner Loan Payable (funding) (Liability)           v4vapp.dhf                     100.00 hive

DESCRIPTION
💬Opening Balance for testing v2 backend


J/E NUMBER: 95801581_3284a7bf2900835e8b0d6d25fd235f38192f2dee_1_real 95801581_3284a
DATE
May 11, 2025 17:03  

ACCOUNT                                                                 DEBIT          CREDIT
----------------------------------------------------------------------------------------------------
Treasury Hive (Asset)                           devtre.v4vapp       50.00 hbd                
    Owner Loan Payable (funding) (Liability)  

In [5]:
for account, balance in balances.items():
    print(f"Balance for {account}:")
    print(balance)

Balance for Customer Deposits Hive (Asset) - Sub: devser.v4vapp:
Balance for Customer Deposits Hive (sub: devser.v4vapp) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: HBD
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-05-12 06:31:03  Send opening balance to Live Server HBD              40.00         0.00        40.00  95817721_ee45a
2025-05-12 07:52:21  Test send back from server to treasury                0.00        10.00        30.00  95819345_1fca0
2025-05-12 08:16:39  Reset for tests hbd                                   0.00        30.00         0.00  95819830_acf8f
2025-05-12 11:38:15  Another test with a private memo                     10.00         0.00        10.00  95823857_6d29b
2025-05-14 16:01:24  Refund to clear all debts to v4vapp-test              0.00        10.00         0.00  95886649_da3f

In [6]:
print(balances["Customer Liability Hive (Liability) - Sub: v4vapp-test"])


Balance for Customer Liability Hive (sub: v4vapp-test) Liability
-----------------------------------------------------------------------------------------------------------------------------

Unit: HBD
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-05-12 11:38:15  Another test with a private memo                      0.00        10.00        10.00  95823857_6d29b
2025-05-14 16:01:24  Refund to clear all debts to v4vapp-test             10.00         0.00         0.00  95886649_da3fc
2025-06-08 11:48:03  Customer to Server deposit: 11.000 HBD fro...         0.00        11.00        11.00  96600833_ef93a
2025-06-08 11:48:33  Fee for Lightning Payment 9,497 sats                  0.24         0.00        10.76      7d09162e24
2025-06-08 11:48:33  Outgoing Lightning Payment 10.032 HBD 9,49...        10.03         0.00         0.73      7d09162e24
2025-06-08 11:49:18  Server to Customer withdrawal: 0.728 HB

In [7]:
print(balances["Fee Income Lightning (Revenue) - Sub: umbrel"])

Balance for Fee Income Lightning (sub: umbrel) Revenue
-----------------------------------------------------------------------------------------------------------------------------

Unit: SATS
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-05-28 10:30:58  Fee for Lightning Payment 2,233 sats                     0          106          106      3e5e24bb3c
2025-06-08 08:36:25  Fee for Lightning Payment 1,113 sats                     0           69          175      430f30bfa0
2025-06-08 11:48:33  Fee for Lightning Payment 9,497 sats                     0          227          402      7d09162e24
2025-06-08 11:55:42  Fee for Lightning Payment 899 sats                       0           65          467      164dbed268
2025-06-08 16:13:32  Fee for Lightning Payment 1,885 sats                     0           82          549      7ebff7c5df
2025-06-09 08:35:35  Fee Lightning v4vapp-test 560 sats              

In [8]:
print(balances["Customer Deposits Hive (Asset) - Sub: devser.v4vapp"])

Balance for Customer Deposits Hive (sub: devser.v4vapp) Asset
-----------------------------------------------------------------------------------------------------------------------------

Unit: HBD
----------
Timestamp            Description                                          Debit       Credit      Balance
----------
2025-05-12 06:31:03  Send opening balance to Live Server HBD              40.00         0.00        40.00  95817721_ee45a
2025-05-12 07:52:21  Test send back from server to treasury                0.00        10.00        30.00  95819345_1fca0
2025-05-12 08:16:39  Reset for tests hbd                                   0.00        30.00         0.00  95819830_acf8f
2025-05-12 11:38:15  Another test with a private memo                     10.00         0.00        10.00  95823857_6d29b
2025-05-14 16:01:24  Refund to clear all debts to v4vapp-test              0.00        10.00         0.00  95886649_da3fc
2025-05-14 16:47:00  More funds treasury to server             